In [7]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate



In [8]:

question_schema = ResponseSchema(
    name="questions",
    description="list of questions about the context with the example: ['What is rag'].",
    type="array(str)"
)


In [9]:

question_response_schemas = [
    question_schema,
]

question_output_parser = StructuredOutputParser.from_response_schemas(question_response_schemas)

In [10]:
question_output_parser

StructuredOutputParser(response_schemas=[ResponseSchema(name='questions', description="list of questions about the context with the example: ['What is rag'].", type='array(str)')])

In [11]:

format_instructions = question_output_parser.get_format_instructions()

In [6]:
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"questions": array(str)  // list of questions about the context with the example: [\'What is rag\'].\n}\n```'

In [12]:
question_generation_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")

In [13]:

bare_prompt_template = "{content}"
bare_template = ChatPromptTemplate.from_template(template=bare_prompt_template)


In [16]:

qa_template = """\
You are a University Professor creating a test for advanced students. For each context, create 5 question that is specific to the context. Avoid creating generic or general questions.

Format the output as JSON with the following keys:
questions:

Format the output as the following:
questions: [
    "Question 1",
    "Question 2"
]

context: {context}
"""

In [17]:
prompt_template = ChatPromptTemplate.from_template(template=qa_template)


In [18]:
prompt_template

ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='You are a University Professor creating a test for advanced students. For each context, create 5 question that is specific to the context. Avoid creating generic or general questions.\n\nFormat the output as JSON with the following keys:\nquestions:\n\nFormat the output as the following:\nquestions: [\n    "Question 1",\n    "Question 2"\n]\n\ncontext: {context}\n'))])

In [ ]:
messages = prompt_template.format_messages(
    context=get_context_for_user_objective(user_prompt),
    format_instructions=format_instructions
)

question_generation_chain = bare_template | question_generation_llm

response = question_generation_chain.invoke({"content" : messages})
questions_dict = question_output_parser.parse(response.content)
print(questions_dict)